# Client Retention

This project..

## Import Libraries

In [10]:
#Data Modulation
import pandas as pd
import numpy as np

#Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

#Machine Learning
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
## Load Data
df = pd.read_csv(C:/Users/erikh/Desktop/Pratt/Machine Learning/Final_Project/customer_retention/Data)

url = 'https://raw.githubusercontent.com/hxchua/datadoubleconfirm/master/datasets/arrivals2018.csv'
df = pd.read_csv(url, error_bad_lines=False)